# Comparing Various Ordinal Embedding Algorithms

In [1]:
import time
import FORTE.utils as utils
from FORTE.algorithms import NuclearNormPGD, FactoredGradient, RankdPGD 
import blackbox
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair

First, to demonstrate the basic idea of embedding from triplet data, we generate a true centered set of points `Xtrue` in two dimensions and using just ordinal information about `Xtruue`, find a set of points satisfying the same ordinal constraints.

A triplet is a three tuple (i,j,k) which represents that X_i is closer to X_j than X_k. We add 10*nd*log(n) such triplets to the training set, `Strain`, and the same number to a test set `Stest`. 

In [2]:
n = 100
d = 2
num_triplets = 20*d*n*np.log(n)

Xtrue = np.random.randn(n, d)
Xtrue = Xtrue - 1. / n * np.dot(np.ones((n, n)),  Xtrue)
Mtrue = np.dot(Xtrue, Xtrue.transpose())
trace_norm = np.trace(Mtrue)

# Train and test set
Strain = utils.triplets(Xtrue, num_triplets)
Stest = utils.triplets(Xtrue, num_triplets)

We now use the `FactoredGradient`  algorithm to find an embedding `Xhat` that agrees with these triplets. A Procrustes transformation is used to align `Xhat` with `Xtrue`. This may take a few seconds to run.

In [ ]:
Xhat = FactoredGradient.computeEmbedding(n, d, Strain,
                                         num_random_restarts=0,
                                         max_num_passes_SGD=16, max_iter_GD=50,
                                         max_norm=1, epsilon=0.01, verbose=False)
print ('Empirical Training loss = {},' 
       'Empirical Test loss = {}').format(utils.empirical_lossX(Xhat, Strain),
                                          utils.empirical_lossX(Xhat, Stest))
                                       

Empirical Training loss = 0.227470141151,Empirical Test loss = 0.22888165038


In [ ]:
Mhat =  NuclearNormPGD.computeEmbedding(n, d,
                                        Strain,
                                        max_iter_GD=10000,
                                        epsilon=0.000001,
                                        trace_norm=trace_norm,
                                        verbose=False)

print ('NuclearNormPGD Empirical Training loss = {},' 
       'Empirical Test loss = {}').format(utils.empirical_lossM(Mhat, Strain),
                                          utils.empirical_lossM(Mhat, Stest))

Mhat =  RankdPGD.computeEmbedding(n, d,
                                  Strain,
                                  max_iter_GD=10000,
                                  epsilon=0.000001,
                                  verbose=True)

print ('RankdPGD Empirical Training loss = {},' 
       'Empirical Test loss = {}').format(utils.empirical_lossM(Mhat, Strain),
                                          utils.empirical_lossM(Mhat, Stest))

NuclearNormPGD Empirical Training loss = 0.014115092291,Empirical Test loss = 0.0304560260586
emp_loss: 0.0 log_loss: 1.19302601681 inner_t: 0 G_norm: 0.387485008483 Delta: 68.1145172287 iter: 1
emp_loss: 0.0 log_loss: 0.749290136429 inner_t: 0 G_norm: 0.336612283961 Delta: 0.00889843984927 iter: 2
emp_loss: 0.0 log_loss: 0.749250550156 inner_t: 0 G_norm: 0.336585435259 Delta: 0.00889693549561 iter: 3
emp_loss: 0.0 log_loss: 0.749210977269 inner_t: 0 G_norm: 0.336558589554 Delta: 0.00889543164242 iter: 4
emp_loss: 0.0 log_loss: 0.74917141776 inner_t: 0 G_norm: 0.336531746848 Delta: 0.00889392828968 iter: 5
emp_loss: 0.0 log_loss: 0.749131871624 inner_t: 0 G_norm: 0.336504907143 Delta: 0.00889242543736 iter: 6
emp_loss: 0.0 log_loss: 0.749092338853 inner_t: 0 G_norm: 0.33647807044 Delta: 0.0088909230855 iter: 7
emp_loss: 0.0 log_loss: 0.74905281944 inner_t: 0 G_norm: 0.336451236743 Delta: 0.00888942123402 iter: 8
emp_loss: 0.0 log_loss: 0.74901331338 inner_t: 0 G_norm: 0.336424406052 De

Finally we use a Procrustes transformation to align `Xtrue` with `Xhat` and plot the resulting embeddings.

In [ ]:
_, Xpro, _ = utils.procrustes(Xtrue, Xhat)
plt.figure(1)
plt.subplot(121); plt.plot(*zip(*Xtrue), marker='o', color='r', ls='')
plt.subplot(122); plt.plot(*zip(*Xpro), marker='o', color='b', ls='')
plt.show()


Now that we see how basic embedding works, we will compare the various embedding algorithms in the `FORTE` package to understand 

- Their accuracy at embedding.
- Their average run-time. 
-  

# Speed Comparisons

In [ ]:
blackbox.set_experiment('SpeedTest')

In [ ]:
# run a test
blackbox.takeoff('Factored Gradient', force=True)
t_FG = time.time()
Xhat_FG = FactoredGradient.computeEmbedding(n, d, Strain,
                                         num_random_restarts=0,
                                         max_num_passes_SGD=16, max_iter_GD=50,
                                         max_norm=1, epsilon=0.01, verbose=False)
blackbox.land()
blackbox.takeoff('Nuclear Norm PGD', force=True)
t_PGD = time.time()
Mhat =  NuclearNormProjected.computeEmbedding(n, d,
                                                 Strain,
                                                 max_iter_GD=10000,
                                                 epsilon=0.000001,
                                                 trace_norm=trace_norm,
                                                 verbose=False)
blackbox.land()


## Get the data from blackbox

In [ ]:
exp = blackbox.get_experiment('SpeedTest')
print exp.list_runs()

runs = []
for key in exp.list_runs():
    run = exp.get_run(key).events
    start_time = t_PGD if key == 'Nuclear Norm PGD' else t_FG
    _ = [r.update({'experiment': key, 'start time': start_time}) for r in run]
    runs += run
    
df = pd.DataFrame(runs)
df['elapsed time'] = df['timestamp'] - df['start time']

In [ ]:
df.head()

In [ ]:
for key in exp.list_runs():
    run = exp.get_run(key).events
    print len(run)

In [ ]:
from altair import Chart
Chart(df).mark_point().encode(
    x='elapsed time', y='emp_loss', color='experiment')